In [1]:
# spotipy modules
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# other libraries
from decouple import config 
import time

In [3]:
# visualisation libraries
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# environment variables
client_id = config('SPOTIPY_CLIENT_ID')
client_secret = config('SPOTIPY_CLIENT_SECRET')
redirect_uri = 'http://google.com/'

In [5]:
# authorization
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

In [6]:
username = 'sidsaxena'

In [7]:
scope = 'playlist-read-private user-library-read user-top-read user-read-recently-played user-follow-read user-read-currently-playing'

In [8]:
try:
    
    token = util.prompt_for_user_token(username=username, scope=scope, client_id=client_id, client_secret= client_secret, redirect_uri=redirect_uri)
    sp = spotipy.Spotify(auth=token)

except:

    print('Token not accessible for user: ', username)


In [9]:
def getUserPlaylists(user):

    name_list = []
    id_list = []
    creator_list = []
    offset=0
    playlists = []

    while True:
        results = sp.user_playlists(user, offset=offset)
        playlists += results['items']

        if results['next'] is not None:
            offset += 50
        else:
            break

    for playlist, _ in enumerate(playlists):
        name_list.append(playlists[playlist]['name'])
        id_list.append(playlists[playlist]['id'])
        creator_list.append(playlists[playlist]['owner']['id'])
    
    creator_id_tuple = tuple(zip(creator_list, id_list))

    playlist_dict = dict(name = name_list, id = id_list, creator = creator_list)
        
    playlist_df = pd.DataFrame(playlist_dict)
    multiple_playlist_dict = list(creator_id_tuple)
    playlist_df.to_csv('{}-playlists.csv'.format(user))
    return playlist_df, multiple_playlist_dict

In [38]:
def analysePlaylist(creator, playlist_id):
    
    tracks = []
    offset = 0
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
        
    while True:
        results = sp.user_playlist_tracks(creator, playlist_id, offset=offset)
        tracks += results['items']

        if results['next'] is not None:
            offset += 50
        else:
            break

    for track in tracks:        # Create empty dict
        playlist_features = {}        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
    
    playlist_df.to_csv('{}-{}.csv'.format(creator, playlist_id))
    return playlist_df

In [11]:
def analysePlaylistsList(playlist_tuple_list):
    """function to analyse multiple playlists"""

    for id, _ in enumerate(playlist_tuple_list):
        playlist_df = analysePlaylist(playlist_tuple_list[id][0], playlist_tuple_list[id][1])

        # playlist_df['playlist'] = PLAYLIST NAME

        if id == 0:
            playlist_tuple_df = playlist_df
        else:
            playlist_tuple_df = pd.concat([playlist_tuple_df, playlist_df], ignore_index=True)

        playlist_tuple_df.to_csv('multiple playlists.csv')
        return playlist_tuple_df

In [12]:
my_playlists_df, my_playlists_list = getUserPlaylists(username)

In [40]:
name = input('Enter Username: ')
playlistid = input('Enter Playlist ID: ')

In [41]:
playlist_df = analysePlaylist(name, playlistid)

In [43]:
playlist_df

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Black Mountain,IV,Space to Bakersfield,1iqd4ySTUyrSPCJDvRu4AR,0.276,0.652,11,-9.918,0,0.0373,0.943000,0.0993,0.1190,74.448,544070,4
1,Daft Punk,Random Access Memories,Motherboard,79koEJRtKOOGJ0VSAF3FMk,0.548,0.707,4,-14.902,0,0.0352,0.922000,0.1050,0.2840,103.009,341653,4
2,Ray LaMontagne,Ouroboros,Part One - Homecoming,4sEsqJFKJrp260Y2qXV1EK,0.280,0.322,6,-14.574,1,0.0364,0.790000,0.0995,0.1150,190.473,509787,3
3,Ray LaMontagne,Ouroboros,"Part One - Hey, No Pressure",5kuFCKfGMFsBenmPfMB3cw,0.434,0.748,1,-7.707,1,0.0522,0.000273,0.1020,0.7030,151.265,394200,4
4,Ray LaMontagne,Ouroboros,Part One - The Changing Man,1G0tirZrwvflyRIg9CQU5q,0.274,0.573,3,-6.895,0,0.0347,0.665000,0.2390,0.2580,126.081,253160,3
5,Ray LaMontagne,Ouroboros,Part One - While It Still Beats,6ED5IYNdnt3ygDCHcbbhDh,0.321,0.550,10,-6.976,1,0.0331,0.323000,0.2380,0.0450,125.896,250333,3
6,Ray LaMontagne,Ouroboros,Part Two - In My Own Way,3S3siqFECq9XTgLHSGF1no,0.547,0.278,6,-10.676,1,0.0296,0.174000,0.1320,0.0615,125.755,396307,4
7,Ray LaMontagne,Ouroboros,Part Two - Another Day,2kakU4Q10eNd0LsLqrv9hF,0.456,0.250,6,-15.872,1,0.0328,0.073000,0.1110,0.0629,127.450,185627,4
8,Ray LaMontagne,Ouroboros,Part Two - A Murmuration of Starlings,7J8Y6y4JTE3TjDdvK1MW6L,0.674,0.213,8,-12.243,0,0.0420,0.920000,0.1030,0.2000,126.023,153267,4
9,Ray LaMontagne,Ouroboros,Part Two - Wouldn't It Make A Lovely Photograph,6OnN5AfKogPnQpEIbItvyX,0.643,0.459,11,-8.532,1,0.0298,0.000556,0.1050,0.1260,125.682,240067,4
